# Global AI Adoption Patterns: Automation vs Augmentation

Analysis of Claude.ai usage patterns across emerging and developed markets using Anthropic's Economic Index dataset.

**Data Source:** [Anthropic Economic Index](https://huggingface.co/datasets/Anthropic/EconomicIndex) (September 2025 Release)

**Date Range:** August 4-11, 2025

## Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Set paths (relative)
DATA_PATH = Path("../data/intermediate")
OUTPUT_PATH = Path("../outputs")
FIGURES_PATH = Path("../figures")

# Create output directories if they don't exist
OUTPUT_PATH.mkdir(exist_ok=True)
FIGURES_PATH.mkdir(exist_ok=True)

# Load data
print("Loading data...")
df_claude = pd.read_csv(DATA_PATH / "aei_raw_claude_ai_2025-08-04_to_2025-08-11.csv")
df_countries = pd.read_csv(DATA_PATH / "iso_country_codes.csv")
df_gdp = pd.read_csv(DATA_PATH / "gdp_2024_country.csv")

print(f"Claude.ai data: {df_claude.shape[0]:,} rows")
print(f"Countries: {df_countries.shape[0]} codes")
print(f"GDP data: {df_gdp.shape[0]} countries")

## Data Exploration

In [ ]:
print("Claude.ai Data Columns:")
print(df_claude.columns.tolist())
print(f"\nUnique geographies: {df_claude['geography'].unique()}")
print(f"Unique facets: {df_claude['facet'].unique()}")
print(f"\nSample rows:")
df_claude.head()

## Classification: Automation vs Augmentation

We classify AI usage patterns into two categories:
- **Automation:** Directive, task execution, script generation (AI does the work)
- **Augmentation:** Feedback loops, iteration, learning, validation (AI assists human work)

In [ ]:
# Map ISO codes to country names
iso_to_country = dict(zip(df_countries['iso_alpha_2'], df_countries['country_name']))
df_claude['country_name'] = df_claude['geo_id'].map(iso_to_country)

# Define classification patterns
AUTOMATION_PATTERNS = ['directive', 'none', 'direct execution', 'script generation', 'automation']
AUGMENTATION_PATTERNS = ['feedback loop', 'task iteration', 'learning', 'validation', 'explain', 'debug']

def classify_interaction(cluster_name):
    """Classify cluster as automation, augmentation, or other."""
    if pd.isna(cluster_name):
        return 'other'
    cluster_lower = str(cluster_name).lower()
    
    for pattern in AUTOMATION_PATTERNS:
        if pattern in cluster_lower:
            return 'automation'
    
    for pattern in AUGMENTATION_PATTERNS:
        if pattern in cluster_lower:
            return 'augmentation'
    
    return 'other'

df_claude['interaction_type'] = df_claude['cluster_name'].apply(classify_interaction)

print("Interaction type distribution:")
print(df_claude['interaction_type'].value_counts())

## Calculate Automation Rates by Country

In [ ]:
# Filter to collaboration facet
df_collab = df_claude[df_claude['facet'] == 'collaboration'].copy()
df_collab['country_name'] = df_collab['country_name'].astype(str).str.strip().str.title()
df_collab = df_collab[df_collab['country_name'].notna()]

# Calculate metrics by country
country_metrics = df_collab.groupby('country_name').agg(
    total_usage=('value', 'sum'),
    automation_volume=('value', lambda x: x[df_collab.loc[x.index, 'interaction_type'] == 'automation'].sum())
).reset_index()

country_metrics['automation_rate'] = (
    country_metrics['automation_volume'] / country_metrics['total_usage'] * 100
).round(1)

country_metrics['augmentation_rate'] = 100 - country_metrics['automation_rate']

# Clean up
country_metrics = country_metrics[country_metrics['total_usage'] > 0]
country_metrics = country_metrics[country_metrics['country_name'] != 'Nan']
country_metrics = country_metrics.sort_values('total_usage', ascending=False)

print("TOP 20 COUNTRIES BY USAGE:")
print("=" * 60)
print(country_metrics[['country_name', 'automation_rate', 'augmentation_rate', 'total_usage']].head(20).to_string(index=False))

## Emerging vs Developed Markets Comparison

In [ ]:
# Define market classifications
EMERGING_MARKETS = [
    'Brazil', 'India', 'China', 'Mexico', 'Indonesia', 'Turkey', 
    'South Africa', 'Nigeria', 'Kenya', 'Egypt', 'Pakistan',
    'Philippines', 'Vietnam', 'Thailand', 'Colombia', 'Argentina',
    'Chile', 'Poland', 'Malaysia', 'Peru', 'Bangladesh'
]

DEVELOPED_MARKETS = [
    'United States', 'United Kingdom', 'Germany', 'France', 'Japan',
    'Canada', 'Australia', 'Singapore', 'South Korea', 'Netherlands',
    'Sweden', 'Switzerland', 'Denmark', 'Norway', 'Finland',
    'Belgium', 'Austria', 'Italy', 'Spain', 'New Zealand', 'Israel'
]

def calculate_weighted_average(data):
    """Calculate volume-weighted automation rate."""
    if data.empty or data['total_usage'].sum() == 0:
        return 0.0
    return (data['automation_volume'].sum() / data['total_usage'].sum()) * 100

# Calculate weighted averages
emerging_data = country_metrics[country_metrics['country_name'].isin(EMERGING_MARKETS)]
developed_data = country_metrics[country_metrics['country_name'].isin(DEVELOPED_MARKETS)]

emerging_rate = calculate_weighted_average(emerging_data)
developed_rate = calculate_weighted_average(developed_data)
difference = emerging_rate - developed_rate

print("MARKET COMPARISON (Volume-Weighted):")
print("=" * 60)
print(f"Emerging Markets: {emerging_rate:.1f}% automation rate")
print(f"  Countries: {len(emerging_data)}")
print(f"\nDeveloped Markets: {developed_rate:.1f}% automation rate")
print(f"  Countries: {len(developed_data)}")
print(f"\nDifference: {difference:.1f} percentage points")

## Visualizations

In [ ]:
import matplotlib.pyplot as plt

# Top 15 countries bar chart
top_15 = country_metrics.head(15).copy()

fig, ax = plt.subplots(figsize=(12, 8))
x = range(len(top_15))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], top_15['automation_rate'], width, label='Automation', color='#e74c3c')
bars2 = ax.bar([i + width/2 for i in x], top_15['augmentation_rate'], width, label='Augmentation', color='#3498db')

ax.set_xlabel('Country')
ax.set_ylabel('Rate (%)')
ax.set_title('Automation vs Augmentation Rates by Country (Top 15 by Usage)')
ax.set_xticks(x)
ax.set_xticklabels(top_15['country_name'], rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 100)

plt.tight_layout()
plt.savefig(FIGURES_PATH / 'automation_vs_augmentation.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_PATH / 'automation_vs_augmentation.png'}")

In [ ]:
# Emerging vs Developed comparison
fig, ax = plt.subplots(figsize=(8, 6))

markets = ['Emerging Markets', 'Developed Markets']
automation_rates = [emerging_rate, developed_rate]
augmentation_rates = [100 - emerging_rate, 100 - developed_rate]

x = range(len(markets))
width = 0.35

ax.bar([i - width/2 for i in x], automation_rates, width, label='Automation', color='#e74c3c')
ax.bar([i + width/2 for i in x], augmentation_rates, width, label='Augmentation', color='#3498db')

ax.set_ylabel('Rate (%)')
ax.set_title('AI Usage Patterns: Emerging vs Developed Markets')
ax.set_xticks(x)
ax.set_xticklabels(markets)
ax.legend()
ax.set_ylim(0, 100)

# Add value labels
for i, (auto, aug) in enumerate(zip(automation_rates, augmentation_rates)):
    ax.text(i - width/2, auto + 1, f'{auto:.1f}%', ha='center', fontsize=10)
    ax.text(i + width/2, aug + 1, f'{aug:.1f}%', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(FIGURES_PATH / 'emerging_vs_developed.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_PATH / 'emerging_vs_developed.png'}")

## Export Results

In [ ]:
# Save country-level results
country_metrics.to_csv(OUTPUT_PATH / 'country_automation_rates.csv', index=False)
print(f"Saved: {OUTPUT_PATH / 'country_automation_rates.csv'}")

# Save summary statistics
summary = pd.DataFrame({
    'metric': ['Emerging Markets Automation Rate', 'Developed Markets Automation Rate', 'Difference (pp)'],
    'value': [f'{emerging_rate:.1f}%', f'{developed_rate:.1f}%', f'{difference:.1f}']
})
summary.to_csv(OUTPUT_PATH / 'summary_statistics.csv', index=False)
print(f"Saved: {OUTPUT_PATH / 'summary_statistics.csv'}")

print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)